# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

Built an investing strategy that selects the 50 stocks with the highest price momentum. From there, calculated recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports



In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks



In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call



In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2813304393551,
 'week52high': 174.12,
 'week52low': 120.95,
 'week52highSplitAdjustOnly': 172.24,
 'week52lowSplitAdjustOnly': 118.78,
 'week52change': 0.38861482633555494,
 'sharesOutstanding': 16533582677,
 'float': 0,
 'avg10Volume': 131881469,
 'avg30Volume': 99241101,
 'day200MovingAvg': 149.07,
 'day50MovingAvg': 158.49,
 'employees': 148914,
 'ttmEPS': 11.52,
 'ttmDividendRate': 0.9050458490505722,
 'dividendYield': 0.005275704873276693,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-24',
 'nextEarningsDate': '2022-01-24',
 'peRatio': 15.222804878808564,
 'beta': 1.3479583676928764,
 'maxChangePercent': 65.21770564798608,
 'year5ChangePercent': 5.820151863656956,
 'year2ChangePercent': 1.6089602454509868,
 'year1ChangePercent': 0.41079400812284,
 'ytdChangePercent': 0.3071321742140537,
 'month6ChangePercent': 0.3696531739819449,
 'month3ChangePercent': 0.09619773065876082,
 'month1ChangePercent': 0.1379776656565507,
 'day30ChangeP

## Parsing Our API Call



In [4]:
data['year1ChangePercent']

0.41079400812284

## Executing A Batch API Call & Building Our DataFrame



In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,154.80,0.359312,N/A
1,AAL,18.40,0.040647,N/A
2,AAP,246.68,0.567448,N/A
3,AAPL,173.20,0.400746,N/A
4,ABBV,124.04,0.203911,N/A
...,...,...,...,...
500,YUM,137.80,0.28066,N/A
501,ZBH,134.15,-0.11364,N/A
502,ZBRA,616.88,0.628173,N/A
503,ZION,65.65,0.558231,N/A


## Removing Low-Momentum Stocks

The investment strategy seeks to identify the 50 highest-momentum stocks in the S&P 500.

Removing all the stocks in DataFrame that fall below this momentum threshold. Then sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.74,2.283921,N/A
1,DVN,47.00,1.974616,N/A
2,FANG,117.90,1.63499,N/A
3,MCHP,91.30,1.545234,N/A
4,MRO,16.70,1.529658,N/A
5,FTNT,320.87,1.504013,N/A
6,NVDA,328.84,1.438218,N/A
7,F,20.28,1.231788,N/A
8,NUE,116.03,1.130215,N/A
9,IT,324.83,1.044616,N/A


## Calculating the Number of Shares to Buy



In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:
That's not a number! 
 Try again:
Enter the value of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Users/harikrishnathallapally/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,81.74,2.283921,239
1,DVN,47.00,1.974616,417
2,FANG,117.90,1.63499,166
3,MCHP,91.30,1.545234,214
4,MRO,16.70,1.529658,1174
5,FTNT,320.87,1.504013,61
6,NVDA,328.84,1.438218,59
7,F,20.28,1.231788,966
8,NUE,116.03,1.130215,168
9,IT,324.83,1.044616,60


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, Built a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Abbreviation `hqm` stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [35]:
hqm_dataframe['One-Month Price Return'].isna().sum()

0

## Calculating Momentum Percentiles

Calculating momentum percentile scores for every stock in the universe. More specifically, Calculating percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`



In [36]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.647525
1      0.221782
2      0.847525
3       0.70297
4      0.431683
         ...   
500    0.540594
501    0.073267
502    0.912871
503    0.835644
504    0.728713
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.685149
1      0.031683
2      0.853465
3      0.954455
4      0.643564
         ...   
500    0.667327
501     0.09901
502    0.794059
503    0.708911
504    0.918812
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.075248
1      0.142574
2      0.918812
3      0.740594
4      0.831683
         ...   
500    0.514851
501    0.091089
502    0.564356
503    0.829703
504    0.718812
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      0.425743
1      0.013861
2       0.70495
3      0.984158
4      0.790099
         ...   
500    0.813861
501    0.087129
502    0.508911
503    0.548515
504    0.809901
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,160.40,N/A,0.353584,0.647525,0.126152,0.685149,-0.134179,0.075248,-0.017107,0.425743,N/A
1,AAL,18.42,N/A,0.040676,0.221782,-0.268922,0.031683,-0.087691,0.142574,-0.184113,0.013861,N/A
2,AAP,243.35,N/A,0.557205,0.847525,0.239814,0.853465,0.188139,0.918812,0.023074,0.70495,N/A
3,AAPL,175.76,N/A,0.409012,0.70297,0.371830,0.954455,0.096269,0.740594,0.136854,0.984158,N/A
4,ABBV,121.83,N/A,0.204776,0.431683,0.105176,0.643564,0.130544,0.831683,0.037782,0.790099,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,135.90,N/A,0.281588,0.540594,0.119228,0.667327,0.024339,0.514851,0.042182,0.813861,N/A
501,ZBH,134.76,N/A,-0.114860,0.073267,-0.186759,0.09901,-0.124750,0.091089,-0.096000,0.087129,N/A
502,ZBRA,624.99,N/A,0.635883,0.912871,0.198083,0.794059,0.041933,0.564356,-0.003542,0.508911,N/A
503,ZION,66.23,N/A,0.541100,0.835644,0.133310,0.708911,0.129186,0.829703,0.000108,0.548515,N/A


## Calculating the HQM Score

Calculating `HQM Score`, which is the high-quality momentum score.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that were calculated.

To calculate arithmetic mean `mean` function from Python's built-in `statistics` module will be used.

In [37]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks



In [38]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy



In [39]:
portfolio_input()

Enter the value of your portfolio:1000000


In [40]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

/Users/harikrishnathallapally/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,160.40,122,0.353584,0.647525,0.126152,0.685149,-0.134179,0.075248,-0.017107,0.425743,0.458416
1,AAL,18.42,1064,0.040676,0.221782,-0.268922,0.031683,-0.087691,0.142574,-0.184113,0.013861,0.102475
2,AAP,243.35,80,0.557205,0.847525,0.239814,0.853465,0.188139,0.918812,0.023074,0.70495,0.831188
3,AAPL,175.76,111,0.409012,0.70297,0.371830,0.954455,0.096269,0.740594,0.136854,0.984158,0.845545
4,ABBV,121.83,160,0.204776,0.431683,0.105176,0.643564,0.130544,0.831683,0.037782,0.790099,0.674257
5,ABC,124.48,157,0.230083,0.469307,0.018193,0.445545,-0.034410,0.277228,-0.061419,0.174257,0.341584
6,ABMD,324.03,60,0.181553,0.388119,0.102336,0.635644,-0.152813,0.063366,-0.117436,0.063366,0.287624
7,ABT,136.55,143,0.256002,0.50297,0.215425,0.821782,0.036238,0.548515,0.065905,0.90495,0.694554
8,ACN,373.52,52,0.509666,0.80198,0.335550,0.934653,0.094147,0.732673,0.007986,0.607921,0.769307
9,ADBE,670.22,29,0.320702,0.594059,0.276034,0.893069,-0.017397,0.332673,-0.020180,0.407921,0.556931


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [41]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File



* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase



In [42]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [43]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output


In [44]:
writer.save()